In [1]:
#import libraries
import numpy as np
import tensorflow as tf
import sys as sy
import struct as st

In [2]:
#input data
fpI = open('/home2/takezawa/imgCode/Out/n1/ewpLH-ImgRes_0.dig', 'rb')
nxI = fpI.read(4)
nxI = st.unpack("i", nxI)
nxI = nxI[0]
nyI = fpI.read(4)
nyI = st.unpack("i", nyI)
nyI = nyI[0]
img_target = []
for _ in range(nxI * nyI):
    data = fpI.read(8)
    data = st.unpack("d", data)
    data = data[0]
    img_target.append(data)
fpI.close()
img_target = np.array(img_target)
target = img_target.reshape(nyI, nxI)
fpIL = open('/home2/takezawa/imgCode/Out/n1/ewpLH-LImgRes_0.dig', 'rb')
nxIL = fpIL.read(4)
nxIL = st.unpack("i", nxIL)
nxIL = nxIL[0]
nyIL = fpIL.read(4)
nyIL = st.unpack("i", nyIL)
nyIL = nyIL[0]
img_low = []
for _ in range(nxIL * nyIL):
    data = fpIL.read(8)
    data = st.unpack("d", data)
    data = data[0]
    img_low.append(data)
fpIL.close()
img_low = np.array(img_low)
low = img_low.reshape(nyIL, nxIL)

In [3]:
#make dataset
width = nxI
height = nyI
num_blocks = (width - 4) * (height - 2)
init_target = 2 * height + 2
wid_block = 6
half_widblock = int(wid_block * 0.5)
hei_block = 3
dim_block = wid_block * hei_block
wid_target_block = 6
hei_target_block = 2
dim_target_block = wid_target_block * hei_target_block
wid_low_block = 6
hei_low_block = 1
dim_low_block = wid_low_block * hei_low_block
arr_data = []
for y in range(height - hei_block + 1):
    for x in range(0, width - wid_block + 1, 2):
        target_block = target[y:y+hei_target_block, x:x+wid_target_block]
        target_block = target_block.reshape(dim_target_block, 1)
        arr_data.extend(target_block)
        low_block = low[y + 2, x:x+wid_low_block]
        low_block = low_block.reshape(dim_low_block, 1)
        arr_data.extend(low_block)
train_data = np.array(arr_data)
train_x = train_data.reshape(int(train_data.size / dim_block), dim_block)
# print(train_x)

d = []
for y in range(hei_block - 1, height, 1):
    for x in range(half_widblock - 1, width - (half_widblock - 1), 2):
        label = target[y, x:x+2]
        d.append(label)
label = np.array(d)
train_label = np.reshape(label, (len(train_x), 2))

In [4]:
#define num
num_input_units = dim_block
num_hidden1_units = 10
num_hidden2_units = 50
num_hidden3_units = 50
num_hidden4_units = 20
num_hidden5_units = 10
learning_rate = 0.0001
num_iter = 10000
num_output = 2

In [6]:
#set parameters, output is 2
x = tf.placeholder(tf.float32, [None, num_input_units])
wscale = 1.0
sigma1 = tf.sqrt(1.0/num_input_units) * wscale
sigma2 = tf.sqrt(1.0/num_hidden1_units) * wscale
sigma3 = tf.sqrt(1.0/num_hidden2_units) * wscale
sigma4 = tf.sqrt(1.0/num_hidden3_units) * wscale
sigma5 = tf.sqrt(1.0/num_hidden4_units) * wscale
sigma6 = tf.sqrt(1.0/num_hidden5_units) * wscale
#hidden layer 1
w_h_1 = tf.Variable(tf.random_normal(shape=[num_input_units, num_hidden1_units], stddev=sigma1))
b_h_1 = tf.Variable(tf.zeros([num_hidden1_units]))

#hidden layer2
w_h_2 = tf.Variable(tf.random_normal(shape=[num_hidden1_units, num_hidden2_units], stddev=sigma2))
b_h_2 = tf.Variable(tf.zeros([num_hidden2_units]))

#hidden layer3
w_h_3 = tf.Variable(tf.random_normal(shape=[num_hidden2_units, num_hidden3_units], stddev=sigma3))
b_h_3 = tf.Variable(tf.zeros([num_hidden3_units]))

#hidden layer4
w_h_4 = tf.Variable(tf.random_normal(shape=[num_hidden3_units, num_hidden4_units], stddev=sigma4))
b_h_4 = tf.Variable(tf.zeros([num_hidden4_units]))

#hidden layer5
w_h_5 = tf.Variable(tf.random_normal(shape=[num_hidden4_units, num_hidden5_units], stddev=sigma5))
b_h_5= tf.Variable(tf.zeros([num_hidden5_units]))

#output layer
w_o = tf.Variable(tf.random_normal(shape=[num_hidden5_units, num_output], stddev=sigma6))
b_o = tf.Variable(tf.zeros([num_output]))

#calculate hidden layer
h_1 = tf.nn.elu(tf.matmul(x, w_h_1) + b_h_1)
h_2 = tf.nn.elu(tf.matmul(h_1, w_h_2) + b_h_2)
h_3 = tf.nn.elu(tf.matmul(h_2, w_h_3) + b_h_3)
h_4 = tf.nn.elu(tf.matmul(h_3, w_h_4) + b_h_4)
h_5 = tf.nn.elu(tf.matmul(h_4, w_h_5) + b_h_5)

#calculate output layer
y = tf.matmul(h_5, w_o) + b_o

t = tf.placeholder(tf.float32, [None, num_output])


loss = tf.losses.mean_squared_error(y, t)
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [ ]:
LOSS = sess.run(tf.sqrt(loss), feed_dict={x:train_x, t:train_label})
for i in range(num_iter + 1):
    sess.run(train_step, feed_dict={x:train_x, t:train_label})
    loss_new = sess.run(tf.sqrt(loss), feed_dict={x:train_x, t:train_label})
    if LOSS > loss_new:
        LOSS = loss_new
        weight_1 = sess.run(w_h_1)
        weight_2 = sess.run(w_h_2)
        weight_3 = sess.run(w_h_3)
        weight_4 = sess.run(w_h_4)
        weight_5 = sess.run(w_h_5)
        weight_out = sess.run(w_o)
        bias_1 = sess.run(b_h_1)
        bias_2 = sess.run(b_h_2)
        bias_3 = sess.run(b_h_3)
        bias_4 = sess.run(b_h_4)
        bias_5 = sess.run(b_h_5)
        bias_out = sess.run(b_o)
    if(i % 1000 == 0):
        print('Step: %d, Loss: %f' % (i, LOSS))
print('END')

In [ ]:
parameter = np.append(weight_1, bias_1)
parameter = np.append(parameter, weight_2)
parameter = np.append(parameter, bias_2)
parameter = np.append(parameter, weight_3)
parameter = np.append(parameter, bias_3)
parameter = np.append(parameter, weight_4)
parameter = np.append(parameter, bias_4)
parameter = np.append(parameter, weight_5)
parameter = np.append(parameter, bias_5)
parameter = np.append(parameter, weight_out)
parameter = np.append(parameter, bias_out)
np.savetxt('parameter_n1_1.txt', parameter);